In [1]:
import torch
import numpy as np

# Load the model parameters from the .pth file
model_path = "./results/model_averaged.pth"
model_state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Initialize global max and min
global_max = -np.inf
global_min = np.inf

# First pass: Find global max and min
for param in model_state_dict.values():
    param_np = param.cpu().numpy()
    param_max = np.max(param_np)
    param_min = np.min(param_np)

    # Update global max and min
    global_max = max(global_max, param_max)
    global_min = min(global_min, param_min)

# Define rescaling and rounding function
def rescale_and_round(value, min_val, max_val, new_min=-128, new_max=127):
    scaled_value = ((value - min_val) / (max_val - min_val)) * (new_max - new_min) + new_min
    return np.round(scaled_value).astype(int)  # Round to nearest integer and cast to int

def decimal_to_signed_binary(value):

    # If the value is negative, calculate two's complement
    if value < 0:
        value = (1 << 8) + value  # Two's complement for negative values

    # Format as an 8-bit binary string
    return f"{value:08b}"

# Specify output file for saving scaled weights and biases
output_file = "./results/model_parameters_scaled_averaged.txt"

# Second pass: Write rescaled parameters to file
with open(output_file, "w") as f:

    f.write(f"Global Max: {global_max:.8f}\n")
    f.write(f"Global Min: {global_min:.8f}\n")

    for name, param in model_state_dict.items():
        # Convert the tensor to a NumPy array
        param_np = param.cpu().numpy()

        # Rescale parameters
        scaled_param_np = rescale_and_round(param_np, global_min, global_max)

        val_binary = decimal_to_signed_binary(val)

        # Write name as 'name = [...]'
        f.write(f"{name} = [\n{{\n")

        # Write shape and global max/min values
        f.write(f"  Shape: {param_np.shape}\n")

        # Write rescaled weights/biases
        f.write("  Scaled Weights/Biases:\n")

        # Check if the parameter is a matrix (2D) or vector (1D) and format accordingly
        if scaled_param_np.ndim == 2:  # Matrix case
            for row in scaled_param_np:
                formatted_row = ", ".join(f"8'sb{val_binary}" for val in row) + ","  # Format each value with comma
                f.write(f"    {formatted_row}\n")
        elif scaled_param_np.ndim == 1:  # Vector case
            formatted_vector = ", ".join(f"8'sb{val_binary}" for val in scaled_param_np) + ","  # Format each value with comma
            f.write(f"    {formatted_vector}\n")
        else:
            f.write(f"    {scaled_param_np}\n")

        # Close the block with braces
        f.write("}\n]\n\n")

print(f"Model parameters have been scaled and saved to {output_file}")


AttributeError: module 'torch' has no attribute 'load'

In [ ]:
!pip install torch